In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
trainData=pd.read_csv('../input/train.csv')
testData=pd.read_csv('../input/test.csv')
def cleaningtrain(data):
    #drop ID
    #data=data.drop(['id'],axis=1)
    #sen and age
    data=pd.merge(data,data['sex and age'].str.split(',',expand=True),left_index=True,right_index=True)
    data.rename(columns={0:'sex',1:'age'},inplace=True)
    data=data.drop(['sex and age'],axis=1)
    data.age, data.sex = np.where(data.sex.str.isalpha()==0, [data.sex, data.age], [data.age, data.sex])
    data.sex=data.sex.str.strip(" ")
    data.sex=data.sex.str.upper()
    data=data.replace({'sex': {'MALE':1,'M':1,'F':0,'FEMALE':0 ,'FEMALLE':0,'MMALE':1,'OTHER':np.NaN,'MM':1,' OTHER':np.NaN}})
    #print(data.sex.unique())
    data.age=data.age.str.strip(" ")
    data=data.replace({'age': {'eleven':11,'seventy seven':77,'fifty':50,'twenty':20,'sixteen':16,'forty':40,'sixx':6,'eight':8,'seven':7,'eighty fivee':85,'six':6,'thirtythree':33}})
    data.age=pd.to_numeric(data.age).round(0)
    #job and living
    jobSet={'private_sector','business_owner','government','parental_leave','unemployed','biz','privattte'}
    data['job_status and living_area']=data['job_status and living_area'].str.strip(" ")
    data['job_status and living_area']=data['job_status and living_area'].str.lower()  
    data=pd.merge(data,data['job_status and living_area'].str.split('?',expand=True),left_index=True,right_index=True)
    data=data.drop(['job_status and living_area'],axis=1)
    data.rename(columns={0:'job',1:'living'},inplace=True)
    #print(data.living.unique())
    data.living, data.job = np.where(data.living.isin(jobSet), [data.job, data.living], [data.living, data.job])
    data=data.replace({'job': {'government':1,'private_sector':2,'business_owner':3,'parental_leave':4 ,'private sector':2,'unemployed':5,'govt.':1,'parental leave':4,'':np.NaN,'private':2, 'null':np.NaN, 'n.a':np.NaN, 'privattte':2,'business owner':3,'biz':3}})
    data=data.replace({'living': {'remote':1,'city':2, 'c':2,'null':np.NaN, 'remotee':1, 'r':1,'':np.NaN}})
    data=data.replace({'high_BP': {'.,':np.NaN,'1':1,'0':0}})
    data=data.replace({'heart_condition_detected_2017': {'N?A':np.NaN,'n.a':np.NaN,'.':np.NaN,'1':1,'0':0}})
    data=data.replace({'married': {'.':np.NaN,'11':1,'1':1,'0':0}})
    data=data.replace({'TreatmentD': {'0+E1860:E1868':np.NaN}})
    data=data.replace({'BMI': {'?':np.NaN,'0+E1860:E1868':np.NaN}})
    data=data.replace({'stroke_in_2018': {'1':1,'0':0,'?':np.NaN,'N?A':1,'.':np.NaN,',':np.NaN,'nuLL':np.NaN}})
    data['smoker_status'] = data['smoker_status'].str.replace(r'quit\W','quit')
    # data['smoker_status'] = data['smoker_status'].str.replace(r'^\W+$','')
    data['smoker_status'] = data['smoker_status'].str.replace(r'\W*','')
    data['smoker_status'] = data['smoker_status'].str.replace(r'\d','')
    data['smoker_status'] = data['smoker_status'].str.replace(r'[_,__,-,?]$','nan')
    data['smoker_status'] = data['smoker_status'].str.replace(r'non$|_nan$|NA','nan')
    data = data.replace({'smoker_status': {'nonsmoker':0,'active_smoker':1,'quit':2,'nan':np.NaN,'':np.NaN}})
    data['smoker_status'] = data['smoker_status'].fillna(-1)
    return data
def cleaningtest(data):
    #drop ID
    #data=data.drop(['id'],axis=1)
    #sen and age
    data=pd.merge(data,data['sex and age'].str.split(',',expand=True),left_index=True,right_index=True)
    data.rename(columns={0:'sex',1:'age'},inplace=True)
    data=data.drop(['sex and age'],axis=1)
    data.age, data.sex = np.where(data.sex.str.isalpha()==0, [data.sex, data.age], [data.age, data.sex])
    data.sex=data.sex.str.strip(" ")
    data.sex=data.sex.str.upper()
    data=data.replace({'sex': {'MALE':1,'M':1,'F':0,'FEMALE':0 ,'FEMALLE':0,'MMALE':1,'OTHER':np.NaN,'MM':1,' OTHER':np.NaN}})
    #print(data.sex.unique())
    data.age=data.age.str.strip(" ")
    data=data.replace({'age': {'eleven':11,'seventy seven':77,'fifty':50,'twenty':20,'sixteen':16,'forty':40,'sixx':6,'eight':8,'seven':7,'eighty fivee':85,'six':6,'thirtythree':33,'fifteen':15,'sixty-seven':67,'sixty three':63,'twenty two':22}})
    data.age=pd.to_numeric(data.age).round(0)
    #job and living
    jobSet={'private_sector','business_owner','government','parental_leave','unemployed','biz','privattte','govt.'}
    data['job_status and living_area']=data['job_status and living_area'].str.strip(" ")
    data['job_status and living_area']=data['job_status and living_area'].str.lower()  
    data=pd.merge(data,data['job_status and living_area'].str.split('?',expand=True),left_index=True,right_index=True)
    data=data.drop(['job_status and living_area'],axis=1)
    data.rename(columns={0:'job',1:'living'},inplace=True)
    #print(data.living.unique())
    data.living, data.job = np.where(data.living.isin(jobSet), [data.job, data.living], [data.living, data.job])
    data=data.replace({'job': {'government':1,'private_sector':2,'business_owner':3,'parental_leave':4 ,'private sector':2,'unemployed':5,'govt.':1,'parental leave':4,'':np.NaN,'private':2, 'null':np.NaN, 'n.a':np.NaN, 'privattte':2,'business owner':3,'biz':3}})
    data=data.replace({'living': {'remote':1,'city':2, 'c':2,'null':np.NaN, 'remotee':1, 'r':1,'':np.NaN}})
    data=data.replace({'BMI': {'.':np.NaN}})
    #print(testData.married.unique())
    #data=data.replace({'high_BP': {'.,':np.NaN,'1':1,'0':0}})
    #data=data.replace({'heart_condition_detected_2017': {'N?A':np.NaN,'n.a':np.NaN,'.':np.NaN,'1':1,'0':0}})
    #data=data.replace({'married': {'.':np.NaN,'11':1,'1':1,'0':0}})
    
    data['smoker_status'] = data['smoker_status'].str.replace(r'quit\W','quit')
    # data['smoker_status'] = data['smoker_status'].str.replace(r'^\W+$','')
    data['smoker_status'] = data['smoker_status'].str.replace(r'\W*','')
    data['smoker_status'] = data['smoker_status'].str.replace(r'\d','')
    data['smoker_status'] = data['smoker_status'].str.replace(r'[_,__,-,?]$','nan')
    data['smoker_status'] = data['smoker_status'].str.replace(r'non$|_nan$|NA','nan')
    data = data.replace({'smoker_status': {'nonsmoker':0,'active_smoker':1,'quit':2,'nan':np.NaN,'':np.NaN,'Na':np.NaN}})
    data['smoker_status'] = data['smoker_status'].fillna(-1)
    #data=data.dropna(subset=['high_BP', 'heart_condition_detected_2017','married','average_blood_sugar','BMI','smoker_status','sex','age','job','living'])
    data=data.drop(columns=[2,3])
    return data

In [ ]:
testData=cleaningtest(testData)
trainData=cleaningtrain(trainData)

In [ ]:
def nanValueTrain(data):
    data=data.dropna(subset=['stroke_in_2018'])
    data['high_BP']=data['high_BP'].fillna(0)
    data['heart_condition_detected_2017']=data['heart_condition_detected_2017'].fillna(0)
    data['married']=data['married'].fillna(1)
    data['average_blood_sugar']=data['average_blood_sugar'].fillna(data['average_blood_sugar'].median())
    data['BMI']=data['BMI'].fillna(data['BMI'].median())
    data['TreatmentA']=data['TreatmentA'].fillna(-1)
    data['TreatmentB']=data['TreatmentB'].fillna(-1)
    data['TreatmentC']=data['TreatmentC'].fillna(-1)
    data['TreatmentD']=data['TreatmentD'].fillna(-1)
    data['sex']=data['sex'].fillna(0)
    data['age']=data['age'].fillna(data['age'].mean())
    data['job']=data['job'].fillna(2)
    data['living']=data['living'].fillna(2)
    return data

def nanValueTest(data):
    #data=data.dropna(subset=['stroke_in_2018'])
    data['high_BP']=data['high_BP'].fillna(0)
    data['heart_condition_detected_2017']=data['heart_condition_detected_2017'].fillna(0)
    data['married']=data['married'].fillna(1)
    data['average_blood_sugar']=data['average_blood_sugar'].fillna(data['average_blood_sugar'].median())
    data['BMI']=data['BMI'].fillna(data['BMI'].median())
    data['TreatmentA']=data['TreatmentA'].fillna(-1)
    data['TreatmentB']=data['TreatmentB'].fillna(-1)
    data['TreatmentC']=data['TreatmentC'].fillna(-1)
    data['TreatmentD']=data['TreatmentD'].fillna(-1)
    data['sex']=data['sex'].fillna(0)
    data['age']=data['age'].fillna(data['age'].mean())
    data['job']=data['job'].fillna(2)
    data['living']=data['living'].fillna(2)
    return data

In [ ]:
train=nanValueTrain(trainData)
test=nanValueTest(testData)

In [ ]:
from sklearn.preprocessing import scale
def standerlize(data):
    data['BMI']=scale(data['BMI'])
    data['average_blood_sugar']=scale(data['average_blood_sugar'])
    data['age']=scale(data['age'])
    return data

In [ ]:
train=standerlize(train)
test=standerlize(test)

In [ ]:
def oneHot(data):
    dfs=pd.get_dummies(data['smoker_status']).add_suffix('_smoker')
    data = data.join(dfs)
    dfj=pd.get_dummies(data['job']).add_suffix('_job')
    data = data.join(dfj)
    data=data.drop(columns=['smoker_status','job'])
    return data
    

In [ ]:
train=oneHot(train)
test=oneHot(test)

In [ ]:
count0, count1 = train['stroke_in_2018'].value_counts()
count1=int(count0*0.80)
stroke0 = train[train['stroke_in_2018']==0]
stroke1 = train[train['stroke_in_2018']==1]
stroke1_up = stroke1.sample(count1, replace=True)
data = pd.concat([stroke0, stroke1_up], axis=0)
print(data['stroke_in_2018'].value_counts())


In [ ]:
from sklearn.model_selection import train_test_split
x = data.drop(['stroke_in_2018','id'],1)
x.describe()
y = data[['stroke_in_2018']]
y.describe()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=20)


In [ ]:
id=test['id']
test=test.drop(columns='id')

In [ ]:
from sklearn.svm import SVC
svc= SVC(gamma='auto',C=2)
svc.fit(x,y)
psvc=svc.predict(test)
plt.hist(psvc)
plt.show()
result=id.to_frame()
result['stroke_in_2018']=psvc
result.to_csv('submission5.csv')

In [ ]:
from sklearn.linear_model import RidgeClassifier
rdg = RidgeClassifier()
rdg.fit(x,y)
prdg=rdg.predict(test)
plt.hist(prdg)
plt.show()
result=id.to_frame()
result['stroke_in_2018']=prdg
result.to_csv('submission6.csv')